In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [19]:
df1 = pd.read_csv("Crime_Data_from_2020_to_Present.csv")
df2 = pd.read_csv("Sheriff_and_Police_Stations.csv")

In [ ]:
df1['DATE OCC'] = pd.to_datetime(df1['DATE OCC'], format='%m/%d/%Y %I:%M:%S %p')

df1['YearMonth'] = df1['DATE OCC'].dt.to_period('M').astype(str)

# Crear el gráfico de dispersión con un control deslizante para cambiar las fechas
fig = px.scatter_mapbox(df1, lat="LAT", lon="LON", color="Crm Cd Desc", hover_name="Crm Cd Desc",
                        hover_data=["DATE OCC", "AREA NAME", "Vict Sex", "Vict Age"],
                        animation_frame="YearMonth", zoom=10, height=600)

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0},clickmode='event+select')

fig.add_scattermapbox(lat=df2['latitude'], 
                       lon=df2['longitude'], 
                       text=df2['name'],
                       hoverinfo='text',
                       mode='markers',
                       marker=dict(size=10, color='blue', opacity=0.8),
                       name='Estaciones de Policía')

fig.show()

In [ ]:
# Convertir la columna 'DATE OCC' a tipo datetime
df1['DATE OCC'] = pd.to_datetime(df1['DATE OCC'], format='%m/%d/%Y %I:%M:%S %p')

# Establecer 'DATE OCC' como índice
df1.set_index('DATE OCC', inplace=True)

# Resamplear los datos por mes y contar el número de crímenes en cada mes
crimes_per_month = df1.resample('M').size()

# Crear el gráfico de la serie de tiempo
plt.figure(figsize=(12, 6))
plt.plot(crimes_per_month, marker='o', linestyle='-')
plt.title('Número de crímenes por mes en Los Ángeles')
plt.xlabel('Fecha')
plt.ylabel('Número de crímenes')
plt.grid(True)
plt.show()

# Mostrar patrones y tendencias utilizando un promedio móvil
crimes_per_month_rolling = crimes_per_month.rolling(window=12).mean()

# Crear el gráfico de la serie de tiempo con la tendencia
plt.figure(figsize=(12, 6))
plt.plot(crimes_per_month, marker='o', linestyle='-', label='Crímenes por mes')
plt.plot(crimes_per_month_rolling, marker='o', linestyle='-', color='red', label='Promedio móvil (12 meses)')
plt.title('Número de crímenes por mes en Los Ángeles con tendencia')
plt.xlabel('Fecha')
plt.ylabel('Número de crímenes')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df1 = df1.sort_values(by='DATE OCC')
train_size = int(len(df1) * 0.8)

#train = df1[:train_size]
#test = df1[train_size:]
#crimes_per_day_train = train.resample('W').size()
#crimes_per_day_test = test.resample('W').size()

original_date = df1.iloc[train_size,1]
original_date = pd.to_datetime(original_date)
new_date = original_date - pd.offsets.MonthEnd(1)
mes_siguiente = new_date + pd.offsets.MonthBegin(1)

# Divide los datos en conjuntos de entrenamiento y prueba
train_graph = df1[:new_date]
test_graph  = df1[mes_siguiente:]

crimes_train = train_graph.resample('ME').size()
crimes_test = test_graph.resample('ME').size()

# Crear el gráfico de la serie de tiempo
plt.figure(figsize=(12, 6))
plt.plot(crimes_train, marker='o', linestyle='-')
plt.plot(crimes_test, marker='o', linestyle='-')
plt.plot([crimes_train.index[-1], crimes_test.index[0]], 
         [crimes_train.iloc[-1], crimes_test.iloc[0]], 
         color='gray', linestyle='--', linewidth=2)
plt.title('Número de crímenes por mes en Los Ángeles')
plt.xlabel('Fecha')
plt.ylabel('Número de crímenes')
plt.grid(True)
plt.show()


In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss
import warnings
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
warnings.filterwarnings("ignore")
diff_1 = crimes_train.diff().dropna()
diff_2 = diff_1.diff().dropna()
diff_3 = diff_2.diff().dropna()

print('Test estacionariedad serie original')
print('-------------------------------------')
adfuller_result = adfuller(crimes_train)
kpss_result = kpss(crimes_train)
print(f'ADF Statistic: {adfuller_result[0]}, p-value: {adfuller_result[1]}')
print(f'KPSS Statistic: {kpss_result[0]}, p-value: {kpss_result[1]}')

print('\nTest estacionariedad para serie diferenciada (order=1)')
print('--------------------------------------------------')
adfuller_result = adfuller(diff_1)
kpss_result = kpss(diff_1.diff().dropna())
print(f'ADF Statistic: {adfuller_result[0]}, p-value: {adfuller_result[1]}')
print(f'KPSS Statistic: {kpss_result[0]}, p-value: {kpss_result[1]}')

print('\nTest estacionariedad para serie diferenciada (order=2)')
print('--------------------------------------------------')
adfuller_result = adfuller(diff_2)
kpss_result = kpss(diff_2.diff().diff().dropna())
print(f'ADF Statistic: {adfuller_result[0]}, p-value: {adfuller_result[1]}')
print(f'KPSS Statistic: {kpss_result[0]}, p-value: {kpss_result[1]}')

print('\nTest estacionariedad para serie diferenciada (order=3)')
print('--------------------------------------------------')
adfuller_result = adfuller(diff_3)
kpss_result = kpss(diff_3.diff().diff().dropna())
print(f'ADF Statistic: {adfuller_result[0]}, p-value: {adfuller_result[1]}')
print(f'KPSS Statistic: {kpss_result[0]}, p-value: {kpss_result[1]}')
warnings.filterwarnings("default")


fig, axs = plt.subplots(nrows=4, ncols=1, figsize=(7, 5), sharex=True)
crimes_train.plot(ax=axs[0], title='Serie original')
diff_1.plot(ax=axs[1], title='Diferenciación orden 1')
diff_2.plot(ax=axs[2], title='Diferenciación orden 2')
diff_3.plot(ax=axs[3], title='Diferenciación orden 3')

El p-value obtenido tras la primera diferenciación es estadísticamente significativo acorde al umbral ampliamente reconocido y aceptado de 0.05. Por lo tanto, la selección más adecuada para el parámetro ARIMA 𝑑 es 3.

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=1, figsize=(12, 10), sharex=True)


plot_acf(diff_3, ax=axs[0], lags=38, alpha=0.05)
axs[0].set_title('Autocorrelación serie diferenciada (order=1)')
axs[0].set_ylabel('ACF')


plot_pacf(diff_3, ax=axs[1], lags=18, alpha=0.05)
axs[1].set_title('Autocorrelación parcial serie diferenciada (order=1)')
axs[1].set_ylabel('PACF')

Acorde a la función de autocorrelación, el valor óptimo para el parámetro 𝑝 es 1. Sin embargo, se va a asignar un valor de 1 para proporcionar un componente autorregresivo al modelo. En cuanto al componente 𝑞 , la función de autocorrelación parcial sugiere un valor de 6.

In [ ]:
res_decompose = seasonal_decompose(crimes_train, model='additive', extrapolate_trend='freq')
res_descompose_diff_3 = seasonal_decompose(diff_3, model='additive', extrapolate_trend='freq')

fig, axs = plt.subplots(nrows=4, ncols=2, figsize=(9, 6), sharex=True)
res_decompose.observed.plot(ax=axs[0, 0])
axs[0, 0].set_title('Serie original')
res_decompose.trend.plot(ax=axs[1, 0])
axs[1, 0].set_title('Tendencia')
res_decompose.seasonal.plot(ax=axs[2, 0])
axs[2, 0].set_title('Estacionalidad')
res_decompose.resid.plot(ax=axs[3, 0])
axs[3, 0].set_title('Residuos')
res_descompose_diff_3.observed.plot(ax=axs[0, 1])
axs[0, 1].set_title('Series diferenciadas (order=3)')
res_descompose_diff_3.trend.plot(ax=axs[1, 1])
axs[1, 1].set_title('Tendencia')
res_descompose_diff_3.seasonal.plot(ax=axs[2, 1])
axs[2, 1].set_title('Estacionalidad')
res_descompose_diff_3.resid.plot(ax=axs[3, 1])
axs[3, 1].set_title('Residuos')
fig.suptitle('Descomposición de la serie original vs serie diferenciada', fontsize=14)
fig.tight_layout()

El patrón recurrente cada 2 meses sugiere una estacionalidad anual, probablemente influenciada por factores vacacionales. El gráfico de ACF respalda aún más la presencia de esta estacionalidad, ya que se observan picos significativos en los lags correspondientes a los intervalos de 2 meses, confirmando la idea de patrones recurrentes.

In [26]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import seaborn as sns

In [ ]:
# Convertir la columna 'DATE OCC' a tipo datetime
df1['DATE OCC'] = pd.to_datetime(df1['DATE OCC'])

# Extraer características numéricas de 'DATE OCC'
df1['Year'] = df1['DATE OCC'].dt.year
df1['Month'] = df1['DATE OCC'].dt.month
df1['Day'] = df1['DATE OCC'].dt.day

# Seleccionar columnas categóricas relevantes y reducir cardinalidad
categorical_columns = ['Crm Cd Desc', 'AREA NAME', 'Vict Sex', 'Vict Descent', 'Premis Desc']
for col in categorical_columns:
    top_categories = df1[col].value_counts().index[:10]
    df1[col] = df1[col].where(df1[col].isin(top_categories), other='Other')

# Definir las características (X) y las variables objetivo (y)
X = df1[['Year', 'Month', 'Day'] + categorical_columns]
# Convertir características categóricas a variables dummy (one-hot encoding)
X = pd.get_dummies(X, drop_first=True)

# Asegurarse de que todas las características sean numéricas
X = X.apply(pd.to_numeric, errors='coerce')

# Eliminar filas con valores NaN generados por la conversión
X = X.dropna()

# Ajustar y a los datos restantes después de la eliminación
y = df1[['LAT', 'LON']].loc[X.index]

# Dividir el DataFrame en conjuntos de entrenamiento y testeo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_lat = LinearRegression()
model_lon = LinearRegression()

model_lat.fit(X_train, y_train['LAT'])
model_lon.fit(X_train, y_train['LON'])

# Predicciones
y_pred_lat = model_lat.predict(X_test)
y_pred_lon= model_lon.predict(X_test)

# Evaluar el modelo
mse_lat = np.sqrt(mean_squared_error(y_test['LAT'], y_pred_lat))
mse_lon = np.sqrt(mean_squared_error(y_test['LON'], y_pred_lon))
print(f"Error cuadrático medio (MSE) para LAT: {mse_lat}")
print(f"Error cuadrático medio (MSE) para LON: {mse_lon}")

# Crear un DataFrame para comparar las predicciones con los valores observados
comparison_df = pd.DataFrame({
    'Fecha': df1.loc[X_test.index, 'DATE OCC'],
    'LAT Observado': y_test['LAT'],
    'LAT Predicho': y_pred_lat,
    'LON Observado': y_test['LON'],
    'LON Predicho': y_pred_lon
})

comparison_df = comparison_df.sort_values('Fecha')

# Configurar el estilo de los gráficos
sns.set(style="whitegrid")

# Crear los gráficos de comparación para LAT y LON
plt.figure(figsize=(14, 7))

# Comparación para LAT
plt.subplot(1, 2, 1)
plt.scatter(comparison_df['LAT Observado'], comparison_df['LAT Predicho'], alpha=0.5)
plt.plot([comparison_df['LAT Observado'].min(), comparison_df['LAT Observado'].max()],
         [comparison_df['LAT Observado'].min(), comparison_df['LAT Observado'].max()],
         color='red', linestyle='--')
plt.title('Comparación de Predicciones vs Valores Observados (LAT)')
plt.xlabel('LAT Observado')
plt.ylabel('LAT Predicho')

# Comparación para LON
plt.subplot(1, 2, 2)
plt.scatter(comparison_df['LON Observado'], comparison_df['LON Predicho'], alpha=0.5)
plt.plot([comparison_df['LON Observado'].min(), comparison_df['LON Predicho'].max()],
         [comparison_df['LON Observado'].min(), comparison_df['LON Predicho'].max()],
         color='red', linestyle='--')
plt.title('Comparación de Predicciones vs Valores Observados (LON)')
plt.xlabel('LON Observado')
plt.ylabel('LON Predicho')

plt.tight_layout()
plt.show()